In [1]:
import pandas as pd
import numpy as np
import torch
import math

In [14]:
exponents = np.arange(1,7,.25)
exponents

array([1.  , 1.25, 1.5 , 1.75, 2.  , 2.25, 2.5 , 2.75, 3.  , 3.25, 3.5 ,
       3.75, 4.  , 4.25, 4.5 , 4.75, 5.  , 5.25, 5.5 , 5.75, 6.  , 6.25,
       6.5 , 6.75])

In [20]:
num_data_points = []
data_list = []
for i in range(len(exponents)):
    num_data_points.append(math.ceil(10**exponents[i]))
    if num_data_points[i] % 2 != 0:
        num_data_points[i] = num_data_points[i] - 1
    tensor = torch.from_numpy(np.random.standard_normal(num_data_points[i]))
    data_list.append(tensor.type(torch.DoubleTensor))
print(num_data_points)

[10, 18, 32, 56, 100, 178, 316, 562, 1000, 1778, 3162, 5624, 10000, 17782, 31622, 56234, 100000, 177828, 316228, 562342, 1000000, 1778280, 3162278, 5623414]


In [21]:
label_list = []
for j in num_data_points:
    temp = torch.from_numpy(np.repeat([0,1], j/2))
    print(len(temp))
    label_list.append(temp.type(torch.DoubleTensor))

10
18
32
56
100
178
316
562
1000
1778
3162
5624
10000
17782
31622
56234
100000
177828
316228
562342
1000000
1778280
3162278
5623414


In [29]:
# time_exp = np.arange(-5,0.5,0.5)
# time_exp
# time_limits = []
# for k in time_exp:
#     time_limits.append(10**k)
time_limit = 1

In [23]:
from square_loss import square_loss
from squared_hinge_loss import squared_hinge_loss
from naive_square_loss import naive_square_loss
from naive_square_hinge_loss import naive_square_hinge_loss
from torch.nn import BCEWithLogitsLoss as logisitic_loss

In [24]:
log_loss = logisitic_loss()
loss_dict ={
    "naive_square": naive_square_loss,
    "naive_square_hinge": naive_square_hinge_loss,
    "square_loss":square_loss,
    "squared_hinge_loss": squared_hinge_loss,
    "logistic_loss": log_loss
}

In [25]:
import time

In [30]:
times_dict_list = []
for loss_name, loss_fun in loss_dict.items():
    print(loss_name)
    for iter in range(10):
        for x in range(len(data_list)):
    #             print(num_data_points[x])
            time_vec = np.zeros(10)
#                 print(iter)
            start_time = time.time()
            if loss_name not in ["logistic_loss"]:
                loss = loss_fun(data_list[x], label_list[x], 1)
            else:
                loss = loss_fun(data_list[x], label_list[x])
            time_vec[iter] = time.time() - start_time
        if (np.median(time_vec) > time_limit):
            print("time too big")
            max_dict = {
                "loss_name": loss_name,
                "max_time": np.max(time_vec),
                "min_time": np.min(time_vec),
                "median_time": np.median(time_vec),
                "data_size": num_data_points[x],
                "time_limit":times
            }
            times_dict_list.append(pd.DataFrame(max_dict,index=[0]))
        


naive_square
time too big
naive_square_hinge
time too big
square_loss
squared_hinge_loss
time too big
logistic_loss


In [31]:
full_dt = pd.concat(times_dict_list)

In [32]:
full_dt

,loss_name,max_time,min_time,median_time,data_size,time_limit
0,naive_square,3.133287,3.106117,3.125668,1000,1.0
0,naive_square_hinge,1.311262,1.253183,1.266807,562,1.0
0,squared_hinge_loss,1.125450,1.069735,1.084981,100000,1.0


In [28]:
full_dt.to_csv("timing_data.csv")